In [ ]:
from Levenshtein import distance
import torch
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

from arithmetic_lm.model import load_model
from arithmetic_lm.tokenizer import CharTokenizer
from arithmetic_lm.constants import PLOTS_DIR, CHECKPOINTS_DIR
from arithmetic_lm.utils import get_carry_str

import warnings

warnings.filterwarnings("ignore")

In [ ]:
tokenizer = CharTokenizer()
stop_token = tokenizer.encode("$")[0]

In [ ]:
# load model
model_name = "trans_dec_6layers_768embd_4head_randsp0.5_rev_ansloss"
ckpt_path = "../checkpoints/addition-generalize-to-longer/trans_dec_6layers_768embd_4head_randsp0.5_rev_ansloss/step1000000-train_loss0.0002-val_loss0.0000.ckpt"
model, hparams = load_model(ckpt_path)
model.to("mps")
model.eval()

In [ ]:
print(hparams["extra_hparams"]["data_format"])
reverse_ops = hparams["extra_hparams"]["data_format"]["reverse_ops"]
reverse_ans = hparams["extra_hparams"]["data_format"]["reverse_ans"]

In [ ]:
def logit_lens(
    model,
    model_name,
    tokenizer,
    prompt_idx: torch.Tensor,
    true_ans: str,
    max_new_tokens: int,
    carry_str: str = None,
    stop_token: str = "$",
):
    """
    Plot unembedded logits from layers
    """

    # shape: (seq_len, n_layers)
    logits = []
    # string, shape (seq_len, n_layers)
    tokens = []

    prompt = prompt_idx.clone()
    pred_str = ""

    for _ in range(max_new_tokens):
        # get logits
        logits_per_layer = get_layer_logits(model, prompt.unsqueeze(0))

        # get top tokens and their logits
        logits_step = []
        tokens_step = []
        for layer in range(model.n_layers):
            top_token_idx = logits_per_layer[layer].argmax().item()
            top_token: str = tokenizer.decode([top_token_idx])
            top_logit: float = logits_per_layer[layer].max().item()
            logits_step.append(top_logit)
            tokens_step.append(top_token)
        logits.append(logits_step)
        tokens.append(tokens_step)

        pred_str += top_token

        if top_token == stop_token:
            break

        # add top token from last layer to prompt
        # top token is the last top_token_idx left from the loop (i know bad practice)
        prompt = torch.cat([prompt, torch.tensor([top_token_idx]).to(prompt.device)])

    # transpose for plotting
    logits = pd.DataFrame(logits).T
    tokens = pd.DataFrame(tokens).T

    prompt_str = tokenizer.decode(prompt_idx.tolist())

    pred_dist = distance(pred_str, true_ans)
    pred_correct = [a == b for a, b in zip(pred_str, true_ans)]

    # plot small carry row on top, and heatmap of logits and add tokens to cells
    # x axis is sequence position, y axis is layer
    # on x axis add final predicted tokens
    scale = 1.5
    fig, ax = plt.subplots(
        figsize=(len(pred_str) / scale, model.n_layers / scale),
    )

    # plot main heatmap
    sns.heatmap(logits, ax=ax, annot=tokens, fmt="", cmap="viridis")
    ax.set_xlabel("Sequence ('c' means a carry was generated)")
    ax.set_ylabel("Layer")
    ax.set_title(
        f"LogitLens for {model_name}\nPrompt: {prompt_str}\nPrediction: {pred_str} (distance: {pred_dist})\nTrue: {true_ans}"
    )

    # set xticklabels to predicted tokens
    ax.set_xticks(np.arange(len(pred_str)) + 0.5)
    xlabels = [f"{a}{'\nc' if c in ('c', 'C') else "\n"}" for a, c in zip(pred_str, carry_str)]
    for i, correct in enumerate(pred_correct):
        if not correct:
            xlabels[i] += f"\n{true_ans[i]}"
    if len(xlabels) < len(pred_str):
        xlabels += pred_str[len(xlabels):]
    ax.set_xticklabels(xlabels)

    # set color of x axis labels to red & add correct pred if it's wrong
    for i, correct in enumerate(pred_correct):
        if not correct:
            ax.get_xticklabels()[i].set_color("red")
            ax.get_xticklabels()[i].set_weight("bold")

    # put boxes around actually predicted token in each column
    tokens_predicted = tokens.apply(lambda x: x == x.iloc[-1], axis=0)
    for i, predicted in enumerate(tokens_predicted.values):
        for j, is_predicted in enumerate(predicted):
            is_true = tokens.iloc[i, j] == true_ans[j]
            if is_predicted or is_true:
                color = {True: "black", False: "red"}[(is_predicted and is_true) or is_true]
                ax.add_patch(
                    plt.Rectangle(
                        (j, i),
                        1,
                        1,
                        fill=False,
                        edgecolor=color,
                        clip_on=False,
                        lw=3,
                    )
                )

    plt.tight_layout()

    # save plot
    plot_path = PLOTS_DIR / "gen_to_longer_rand_spaces" / f"{model_name}_logitlens.png"
    print(f"Saving plot to {plot_path}")
    # plt.savefig(plot_path, dpi=300)
    plt.show()

    return logits, tokens, pred_str